In [ ]:
BATCH_SIZE = 64
MEMORY_SIZE = 50000
MEMORY_SIZE_MIN = 1000

EPSILON = 0.99
EPSILON_DECAY = 0.999975
EPSILON_MIN = 0.01
DISCOUNT_FACTOR = 0.1

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

LEARNING_RATE = 0.001
LEARNING_RATE_DECAY = 0.99975
LEARN_EPOCH = 1000

UPDATE_TIME = 5

In [ ]:
class Agent:

    def __init__(self, state_size, action_size):

        self.state_size = state_size
        self.action_size = action_size

        # memory
        self.batch_size = BATCH_SIZE
        self.memory_size = MEMORY_SIZE
        self.memory_size_min = MEMORY_SIZE_MIN
        self.memory = deque(maxlen = self.memory_size)

        # epsilon
        self.epsilon = EPSILON
        self.epsilon_decay = EPSILON_DECAY
        self.epsilon_min = EPSILON_MIN
        self.discount_factor = DISCOUNT_FACTOR

        self.device = DEVICE

        # 인공신경망 model, 타깃신경망 target_model
        self.learning_rate = LEARNING_RATE
        self.learning_rate_decay = LEARNING_RATE_DECAY
        self.learn_epoch = LEARN_EPOCH
        self.loss = nn.MSELoss()
        self.loss_list = []
        self.model = NeuralNet(self.action_size).to(self.device)
        self.target_model = NeuralNet(self.action_size).to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr = self.learning_rate, eps = 1e-4)
        self.lr_scheduler = lr_scheduler.CyclicLR(self.optimizer, base_lr=0.00005,
                                              step_size_up=5, max_lr=0.01,
                                              gamma=0.5, mode='exp_range')

        # 일정 time step(한 episode)이 지나면 target 신경망을 인공신경망으로 대체
        self.update_time = UPDATE_TIME
        self.update_target_model()

    # 인공신경망의 파라미터 -> target model의 파라미터
    def update_target_model(self):
        self.target_model.load_state_dict(self.model.state_dict())

    # 엡실론 - 탐욕 정책
    def get_action(self, state):

        random_prob = np.random.rand()
        if random_prob <= self.epsilon:
          action_idx = np.random.choice(range(self.action_size))

        else:
          self.model.eval()

          with torch.no_grad():
            state = torch.tensor(state, dtype = torch.float32).to(self.device)
            q_value = self.model(state.unsqueeze(0).unsqueeze(0))

            _, action_idx = torch.max(q_value, dim = 1)
            action_idx = action_idx.item()

        return action_idx

    def append_sample(self, state, action, next_state, reward, exploded, done):
        # done: 게임이 끝났으면 True
        # 이때 reward를 다르게 줌
        self.memory.append((state, action, next_state, reward, exploded, done))

    def train_model(self):

        # 엡실론 업데이트
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

        mini_batch = random.sample(self.memory, self.batch_size)
        states, actions, next_states, rewards, explodeds, dones = zip(*mini_batch)

        states = torch.tensor(states, dtype = torch.float32).unsqueeze(1).to(self.device)
        actions = torch.tensor(actions, dtype = torch.int64).to(self.device)
        next_states = torch.tensor(next_states, dtype = torch.float32).unsqueeze(1).to(self.device)
        rewards = torch.tensor(rewards, dtype = torch.float32).to(self.device).reshape(-1, 1)
        explodeds = torch.tensor(explodeds, dtype = torch.int64).to(self.device).reshape(-1, 1)
        dones = torch.tensor(dones, dtype = torch.int64).to(self.device).reshape(-1, 1)


        self.model.train()
        self.target_model.eval()

        # 현재 상태에 대한 action별 큐함수(인공신경망의 예측)
        predicts = self.model(states)

        with torch.no_grad():

            target_predict, _ = torch.max(self.target_model(next_states), dim = 1)
            target_pred = target_predict.reshape(-1, 1)

        # 벨만 최적 방정식
        targets = rewards + (1 - dones) * self.discount_factor * target_pred
        targets = targets.flatten()

        target_q_values = copy.deepcopy(predicts.detach())
        target_q_values[range(BATCH_SIZE), actions] = targets

        loss = self.loss(predicts, target_q_values)
        self.loss_list.append(loss.item())

        # 가중치 업데이트
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()